# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,2685750,Östersund,99.0,100.00,2721.0,267.45,272.04,272.15,1013.0,100.0,NaN,NaN,NaN,NaN
1,1,3831208,Qaanaaq,100.0,59.00,26381.0,258.75,263.81,263.81,1010.0,81.0,1010.0,1010.0,86.349741,82.118957
2,2,5554428,Ketchikan,1.0,60.00,27415.0,269.83,274.15,274.15,1011.0,80.0,NaN,NaN,-52.936276,16.030378
3,3,2206939,Bluff,56.0,11.62,28537.0,280.11,285.37,285.37,1025.0,80.0,NaN,NaN,-32.749052,85.423768
4,4,1525798,Balkhash,0.0,70.00,25215.0,244.26,252.15,252.15,1037.0,83.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,1805518,Jining,0.0,6.00,27508.0,266.40,275.08,275.08,1031.0,69.0,1031.0,1026.0,NaN,NaN
559,559,1856434,Murakami,75.0,360.00,27915.0,270.62,279.15,279.15,1013.0,87.0,NaN,NaN,NaN,NaN
560,560,2171069,Colac,100.0,5.81,29935.0,298.82,297.04,300.93,1017.0,45.0,NaN,NaN,NaN,NaN
561,561,3698608,Celendín,100.0,118.00,28458.0,284.25,284.58,284.58,1016.0,92.0,1016.0,748.0,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [4]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 4)

# fig.add_layer(heat_layer)

# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
406,406,2137773,Vao,100.0,84.0,29889.0,296.61,298.89,298.89,1008.0,85.0,1008.0,1002.0,-70.289378,153.514754
192,192,3655185,Nueva Loja,100.0,341.0,29432.0,297.96,294.32,294.32,1015.0,97.0,1015.0,981.0,84.100938,-105.723439
19,19,1282256,Hithadhoo,100.0,325.0,30123.0,301.98,301.23,301.23,1012.0,76.0,1012.0,1012.0,-13.205522,74.634486
55,55,2063036,Port Lincoln,100.0,36.0,29467.0,290.75,294.67,294.67,1008.0,79.0,1008.0,1005.0,-4.557161,18.317279
451,451,2123814,Leningradskiy,100.0,102.0,26049.0,255.39,260.49,260.49,1031.0,95.0,1031.0,1030.0,-69.820904,13.774080
271,271,2125693,Evensk,100.0,352.0,25848.0,251.77,258.48,258.48,1017.0,86.0,1017.0,1010.0,-76.673095,154.728901
419,419,4031637,Lavrentiya,100.0,43.0,26319.0,250.39,263.19,263.19,1011.0,90.0,1011.0,1010.0,44.143543,85.389637
545,545,359796,Suez,67.0,166.0,29187.0,289.99,291.87,291.87,1013.0,70.0,1013.0,1011.0,28.460135,57.903575
269,269,2126199,Cherskiy,100.0,261.0,24886.0,243.42,248.86,248.86,1027.0,89.0,1027.0,1027.0,-69.862807,-8.581957
437,437,8079926,Labuhan,100.0,291.0,30198.0,304.04,301.98,301.98,1008.0,71.0,1008.0,1008.0,16.622654,99.803433


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City"})
hotel_df

,index,City,Latitude,Longitude,Hotel Name
0,406,Vao,-70.289378,153.514754,
1,192,Nueva Loja,84.100938,-105.723439,
2,19,Hithadhoo,-13.205522,74.634486,
3,55,Port Lincoln,-4.557161,18.317279,
4,451,Leningradskiy,-69.820904,13.774080,
5,271,Evensk,-76.673095,154.728901,
6,419,Lavrentiya,44.143543,85.389637,
7,545,Suez,28.460135,57.903575,
8,269,Cherskiy,-69.862807,-8.581957,
9,437,Labuhan,16.622654,99.803433,


In [7]:
# #the latitudes and longitudes I saved from citipy were not getting along with the google api at this point, so I decided
#     #to get the place_id from the api and then get google's preferred coordinates from their own native api in order to 
#     #get to the hotel data.

# #get place_id for city
# placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         placeid_params = {
#         "input": row['City'],
#         "inputtype": "textquery",
#         "key": gkey,
#     }
#         ID = requests.get(placeid_url, params=placeid_params).json()
#         ID = (ID["candidates"][0]["place_id"])
#         print(ID)
    

In [8]:
# #get coordinates from place_id
# coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         coordinate_params = {
#         "place_id": ID,
#         "key": gkey,
#     }
# #     assemble url and make API request
#         coords = requests.get(coordinate_url, params=coordinate_params).json()
#         lat = coords['result']['geometry']['location']['lat']
#         lng = coords['result']['geometry']['location']['lng']
#         coords = f'{lat},{lng}'
#         print(coords)

In [9]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = coords_req['result']['address_components'][2]['long_name']
        print(f'Country = {Country}')
    except:
        print(f'County = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
    except:
        results = "n/a"

    
    
    try:
        print(f"Closest hotel to {City} is {results}.")
    except (KeyError, IndexError):
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
        
    print("------------")

City = Vao
Country = Vanuatu
Retrieving Results for hotels near Vao:
Closest hotel to Vao is Tamtam Bungalows.
------------
City = Nueva Loja
Country = Sucumbios
Retrieving Results for hotels near Nueva Loja:
Closest hotel to Nueva Loja is Limoncocha Great Hotel.
------------
City = Hithadhoo
Country = South Province
Retrieving Results for hotels near Hithadhoo:
Closest hotel to Hithadhoo is Scoop Guest House.
------------
City = Port Lincoln
Country = Australia
Retrieving Results for hotels near Port Lincoln:
Closest hotel to Port Lincoln is Port Lincoln Hotel.
------------
City = Leningradskiy
Country = Tsentralnyy administrativnyy okrug
Retrieving Results for hotels near Leningradskiy:
Closest hotel to Leningradskiy is The Ritz-Carlton, Moscow.
------------
City = Evensk
Country = Magadan Oblast
Retrieving Results for hotels near Evensk:
Closest hotel to Evensk is n/a.
------------
City = Lavrentiya
Country = Chukotka Autonomous Okrug
Retrieving Results for hotels near Lavrentiya:
C

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
